# Lets make an interactive web application from scratch

First we have to import some dependencies 

In [ ]:
import sys
sys.path.append('..')
import urbanpy as up
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas()

## Downloading urban data

This notebook is going to use the Lima's metropolitan area as an example. now we are going to download Lima province & Callao province limits and merge them to obtain Lima's Metropolitan area.

In [ ]:
manta = up.download.nominatim_osm('Manta, Ecuador')

In [ ]:
manta.plot()

It is useful to have a uniform spatial unit in order to apply algorithms and visualize differents cities data. UrbanPy lets your easily leverage all the power of Uber H3 package to partition the city in uniform hexagons.

In [ ]:
hex_manta = up.geom.gen_hexagons(resolution=9, city=manta)

In [ ]:
hex_manta.head()

In [ ]:
hex_manta.shape

In [ ]:
hex_manta.plot()

### Downloading high resolution population data

Using the `download.hdx_fb_population` function you can download high resolution population data, please be patient this may take a while ...

In [ ]:
ecu_elderly_pop = up.download.hdx_fb_population('ecuador', 'elderly')

In [ ]:
manta_elderly_pop = up.geom.filter_population(ecu_elderly_pop, manta)

In [ ]:
manta_elderly_pop.head()

### Unify data layers

Lets get the population per hexagon

In [ ]:
hex_manta_gdf = up.geom.merge_shape_hex(hex_manta, manta_elderly_pop, how='inner', op='intersects', agg={'population': 'sum'})

In [ ]:
hex_manta_gdf.head()

In [ ]:
hex_manta_gdf.plot(column='population', legend=True, missing_kwds={'color': 'grey'})
plt.show()

**Validation**: The difference between original population total and downscaled population total must be a small number

In [ ]:
(manta_elderly_pop['population'].sum(), hex_manta_gdf['population'].round(0).sum(), 
 manta_elderly_pop['population'].sum() - hex_manta_gdf['population'].round(0).sum())

## Downloading Points of Interest (PoI) data

### Health facilities

In [ ]:
health_facilities = up.download.overpass_pois(bounds=manta.total_bounds, facilities='health')

In [ ]:
health_facilities.head()

In [ ]:
health_facilities['poi_type'].value_counts()

### Measuring accesibility

#### Accesibility to hospital and clinics

 To find the nearest food facility to each hexagon we are going to use a neighborhood search algorithm (lower precision but significant time savings)

In [ ]:
clinics_and_hospitals = health_facilities[health_facilities['poi_type'].isin(['clinic', 'hospital'])]

In [ ]:
# Get Hexagon centroid latitude and longitude
hex_manta_gdf['lon'] = hex_manta_gdf.geometry.centroid.x
hex_manta_gdf['lat'] = hex_manta_gdf.geometry.centroid.y

In [ ]:
# Get distance and indexes from nearest points 
dist, ind = up.utils.nn_search(
    tree_features = clinics_and_hospitals[['lat', 'lon']].values, # Point of Interest
    query_features = hex_manta_gdf[['lat', 'lon']].values, # Hexagon
    metric='manhattan' # Distance metric
)

In [ ]:
# Add nearest health facility index to the hexagon gdf
hex_manta_gdf['nearest_health_facility_ix'] = ind

Now we are going to find the route distance and duration using an OSRM local server

In [ ]:
# start server
up.routing.start_osrm_server('ecuador', 'south-america', 'foot')

First we are going to find the route distance and duration for one hexagon

In [ ]:
origin = hex_manta_gdf.geometry.centroid[0] # Origin Point

In [ ]:
destination = clinics_and_hospitals.iloc[hex_manta_gdf['nearest_health_facility_ix'][0]].geometry # Destination Point

In [ ]:
# Distance (meters), Duration (seconds)
up.routing.osrm_route(origin, destination)

Now we apply this function to all the origin points (hexagons) and their corresponding destination

In [ ]:
# Distancia y duración del viaje a pie
dist_dur = hex_manta_gdf.progress_apply(
    lambda row: up.routing.osrm_route(
        origin=row.geometry.centroid, 
        destination=clinics_and_hospitals.iloc[row['nearest_health_facility_ix']].geometry,
    ),
    result_type='expand', axis=1
)

In [ ]:
dist_dur

In [ ]:
hex_manta_gdf['distance_health_facility'] =  dist_dur[0] / 1000 # meters to km

In [ ]:
hex_manta_gdf['duration_health_facility'] =  dist_dur[1] / 60 # seconds to minutes

Visualize results

In [ ]:
hex_manta_gdf.plot(column='duration_health_facility', cmap='magma_r', legend=True, missing_kwds={'color': 'grey'})
plt.show()

In [ ]:
up.routing.stop_osrm_server('ecuador', 'south-america', 'foot')

# Using Dash to create an interactive webapp

Import Dash dependencies

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In this example we are using JupyterDash to show our webapp within jupyter lab or a notebook.

In [ ]:
from jupyter_dash import JupyterDash

In [ ]:
app = JupyterDash(name='UrbanPy Dashboard')

In [ ]:
VARIABLE_OPTIONS = [
    {'label': 'Duración del viaje', 'value': 'duration_health_facility'},
    {'label': 'Población mayor de 60', 'value': 'population'}
]

In [ ]:
app.layout = html.Div([
    html.H3('Analisis de Acceso a Comida para Población Mayor en Manta, Ecuador'),
    dcc.Dropdown(id='variable', value='population', options=VARIABLE_OPTIONS),
    dcc.Graph(id="map")
])

In [ ]:
@app.callback(Output('map', 'figure'),
              [Input('variable', 'value')])
def update_map(var):
    return up.plotting.choropleth_map(gdf=hex_manta_gdf, color_column=var)

In [ ]:
app.run_server(mode='inline')

Thanks for viewing this example notebook. You can find the urbanpy documentation [here](https://ingenieriaup.github.io/urbanpy/).

Useful resources:

1. [Dash Getting Started](https://dash.plotly.com/layout)
2. [Dash Make your Webapp public available](https://dash.plotly.com/deployment)